In [44]:
import requests
import json
import pandas as pd

In [45]:
df_category = pd.read_csv('category_info.csv')
df_category.head()

,category,category_url
0,Mẹ & Bé,https://shopee.vn/M%E1%BA%B9-B%C3%A9-cat.11036194
1,Điện Thoại & Phụ Kiện,https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...
2,Máy Tính & Laptop,https://shopee.vn/M%C3%A1y-T%C3%ADnh-Laptop-ca...
3,Đồ Chơi,https://shopee.vn/%C4%90%E1%BB%93-Ch%C6%A1i-ca...
4,Máy Ảnh & Máy Quay Phim,https://shopee.vn/M%C3%A1y-%E1%BA%A2nh-M%C3%A1...


In [46]:
df_category['category_id'] = df_category['category_url'].str.extract(r'cat\.(\d+)')
df_category.head()

,category,category_url,category_id
0,Mẹ & Bé,https://shopee.vn/M%E1%BA%B9-B%C3%A9-cat.11036194,11036194
1,Điện Thoại & Phụ Kiện,https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%...,11036030
2,Máy Tính & Laptop,https://shopee.vn/M%C3%A1y-T%C3%ADnh-Laptop-ca...,11035954
3,Đồ Chơi,https://shopee.vn/%C4%90%E1%BB%93-Ch%C6%A1i-ca...,11036932
4,Máy Ảnh & Máy Quay Phim,https://shopee.vn/M%C3%A1y-%E1%BA%A2nh-M%C3%A1...,11036101


In [47]:
# Hàm để tìm dictionary theo catid
def find_dictionary_by_catid(catid, data):
  for item in data:
    if item.get('catid') == catid:
      return item
  return None  # Trả về None nếu không tìm thấy catid trong danh sách

def get_list_category_children(catid_lv1, cat_name, data_category_level_2):
  list_of_category_children = []
  list_of_category_children.append({
    'category_id':catid_lv1,
    'category_level':1,
    'category_level_1':cat_name,
    'display_name':cat_name,
  })
  for cat in data_category_level_2:
    list_of_category_children.append({
    'category_id':cat['catid'],
    'category_level':cat['level'],
    'category_level_1':cat_name,
    'display_name':cat['display_name'],
    })
  return list_of_category_children

In [48]:
url = "https://shopee.vn/api/v4/pages/get_category_tree"

headers = {
  'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0"
}

r = requests.get(url, headers = headers)
if r.status_code == 200:
  data_category_tree = r.json()["data"]
else:
  print('Lỗi requests')

In [49]:
list_all_category = []
for idx,row in df_category.iterrows():
  # Tìm dictionary có catid nhất định
  result = find_dictionary_by_catid(int(row["category_id"]), data_category_tree['category_list'])
  if result:
    data_category_level_2 = result["children"]
  else:
    pass
  list_all_category += get_list_category_children(row["category_id"], row["category"], data_category_level_2)

In [50]:
df_all_cat = pd.DataFrame(list_all_category)
df_all_cat

,category_id,category_level,category_level_1,display_name
0,11036194,1,Mẹ & Bé,Mẹ & Bé
1,11036195,2,Mẹ & Bé,Đồ dùng du lịch cho bé
2,11036204,2,Mẹ & Bé,Đồ dùng ăn dặm cho bé
3,11036213,2,Mẹ & Bé,Phụ kiện cho mẹ
4,11036217,2,Mẹ & Bé,Chăm sóc sức khỏe mẹ
...,...,...,...,...
309,11037007,2,Thiết Bị Điện Gia Dụng,Quạt & Máy nóng lạnh
310,11037016,2,Thiết Bị Điện Gia Dụng,Thiết bị chăm sóc quần áo
311,11037023,2,Thiết Bị Điện Gia Dụng,Khác
312,11111623,2,Thiết Bị Điện Gia Dụng,"Máy xay, ép, máy đánh trứng trộn bột, máy xay ..."


In [51]:
df_all_cat.to_csv('all_category_levels_info.csv', index = False)